In [18]:
import numpy as np
import pandas as pd
from pomegranate import *
import random
import os
import itertools

### Methods

In [37]:
def init(K,X):
    """This method initializes the models for EM
    
    K: Number of clusters
    X: Data
    
    Return: x_k, models,alpha_k, indices_array, CML
    """
    LENGTH, DIMENSION = X.shape
    models = []
    x_k = [[] for i in range(K)] #initialize K empty data arrays
    alpha_k = []
    indices_array = [[] for i in range(K)]
    print("**************** K =", K ,"************************")


    # Sequences for initial CL Multinet Estimation
    # Make K subsets of data
    for i in range(LENGTH):
        random_integer = random.randint(0,K-1)
        x_k[random_integer].append(X[i])
        indices_array[random_integer].append(i)

    for i in range(K):
        print("Length of model",i+1,":",len(x_k[i]))
        alpha_k.append(len(x_k[i])/LENGTH)
        model = BayesianNetwork.from_samples(x_k[i],algorithm='chow-liu') 
        models.append(model)

    print("Initial Model Structures",alpha_k)
    for model in models:
        print(model.structure)

    print("Initial Alphas",alpha_k)

    CML = 0

    for i in range(K):
        x = x_k[i]
        model = models[i]
        CML+=sum(np.log(model.probability(x)))+len(x)*np.log(alpha_k[i])
    print("Initial CML",CML)
    
    return x_k,models,alpha_k,indices_array,CML
           
def e_step(K,X,x_k,models,alpha_k,indices_array):
    """This method performs the E step in EM for the mth iteration
    
    K: Number of clusters
    X: Data
    x_k: Previously classified data in the (m-1)th step
    models: models from the (m-1)th step
    alpha_k: alphas from the previous step
    indices_array: current indices of the original data (for each cluster)
    
    Return: x_k,models, alpha_k,indices_array
    """
    x_k_temp = [[] for i in range(K)] #initialize K empty data arrays for C step (assign)
    indices_array = [[] for i in range(K)]

    #E Step: Calculate each point's posterior probability for K clusters (trees)
    for idx_first,x in enumerate(X):
        model_prob = []
        for idx,model in enumerate(models): # K trees
            try:
                model_prob.append(model.probability(x))
            except KeyError: #if a point doesn't exist in a tree, then the probability is zero
                model_prob.append(0)
        total = [a*b for a,b in zip(model_prob,alpha_k)]
        max_prob_idx = total.index(max(total)) #return index of the max posterior probability
        x_k_temp[max_prob_idx].append(x)
        indices_array[max_prob_idx].append(idx_first)

    #C step: Assign data-points to the trees that maximize their posterior probability
    x_k = x_k_temp
    alpha_k = [len(x_k[i])/LENGTH for i in range(K)]
    models = []
    for j in range(K):
        model = BayesianNetwork.from_samples(x_k[j],algorithm='chow-liu') 
        models.append(model)
        
    return x_k,models, alpha_k,indices_array

def m_step(K,x_k, models, alpha_k,CML):
    """This method performs the M step in EM for the mth iteration
    
    K: Number of clusters
    x_k: Previously classified data in the (m-1)th step
    models: models from the (m-1)th step
    alpha_k: alphas from the previous step
    CML: Classification Maximum Likelihood    
    
    Return: x_k,models, alpha_k,indices_array
    """
    #M step: Calculate the CML criterion and re-estimate parameters
    init_CML = CML
    CML=0
    for j in range(K):
        x = x_k[j]
        model = models[j]
        CML+=sum(np.log(model.probability(x)))+len(x)*np.log(alpha_k[j])
    return CML, models

# def s_step(K,X):

#     LENGTH, DIMENSION = X.shape
    
#     #S Step
#     x_k = [[] for i in range(K)] #initialize K empty data arrays
#     #S step: Assign data-points randomly
#     for j in range(LENGTH):
#         x_k[random.randint(0,K-1)].append(X[j])
#     alpha_k = [len(x_k[k])/LENGTH for k in range(K)]
#     models = []
#     for j in range(K):
#         model = BayesianNetwork.from_samples(x_k[j],algorithm='chow-liu') 
#         models.append(model)

#     #M step: Calculate the CML criterion and re-estimate parameters
#     init_CML = CML
#     CML=0
#     for j in range(K):
#         x = x_k[j]
#         model = models[j]
#         CML+=sum(np.log(model.probability(x)))+len(x)*np.log(alpha_k[j])
#     print("New CML is:", CML)   


def save(indices_array, path):
    """This method saves the clustered data
    
    indices_array: array of the indices (of the original data) for each cluster
    path: path to save to, sample: '/Users/akankshitadash/Desktop/Bayesian Networks1/RPF_chrE/'
    Directory should already exist, and contain subdirectories of Genes/ and AccNum/
    
    Return: x_k,models, alpha_k,indices_array
    """
    for idx,indices in enumerate(indices_array):
        genes=[]
        acc_nums=[]
        for index in indices:
            genes.append(df.iloc[index]['GeneName'])
            acc_nums.append(df.iloc[index]['AccNum'])
        print(len(indices),len(genes),len(acc_nums))
    #             os.mkdir('/Users/akankshitadash/Desktop/Bayesian Networks/'+str(len(indices_array)))
    #             os.mkdir('/Users/akankshitadash/Desktop/Bayesian Networks/'+str(len(indices_array))+'/Genes/')
    #             os.mkdir('/Users/akankshitadash/Desktop/Bayesian Networks/'+str(len(indices_array))+'/AccNums/')
        with open(path+str(len(indices_array))+'/Genes/Gene'+str(idx+1)+'.txt','w') as f:
            for gene in genes:
                f.write("%s\n" % gene)
        with open(path+str(len(indices_array))+'/AccNums/AccNum'+str(idx+1)+'.txt','w') as f:
            for acc_num in acc_nums:
                f.write("%s\n" % acc_num)
    
def em(K,X,path):    
    """This method performs EM
    
    K: Number of clusters
    X: Discrete data
    path: path to save to
    Return: None
    """
    
    x_k,models,alpha_k,indices_array,CML = init(K,X) #initialize K models
    prev_CML = CML
    
    for i in range(100): #start with 100 iterations of EM
        x_k, models, alpha_k,indices_array = e_step(K,X,x_k,models,alpha_k,indices_array)
        CML, models = m_step(K,x_k, models, alpha_k,CML)
        if(prev_CML==CML):
            break
        else:
            prev_CML = CML
            print("CML is",CML)
    save(indices_array,path)

### Read RPF Data

In [20]:
df = pd.read_csv('AdjustedRPKMOutput/RPF_chrE.txt',sep='\t')

In [21]:
df.head(5)

,AccNum,GeneName,cdReads0,cdRPKM0,cdReads1,cdRPKM1,cdReads2,cdRPKM2,cdReads3,cdRPKM3,cdReads4,cdRPKM4
0,NM_017847,ODR4,93.0,16.468792,62.0,13.189469,49.0,14.172717,37.0,9.403085,39.0,10.487249
1,NM_001143986,TLE6,4.0,0.562465,2.0,0.337849,2.0,0.459350,1.0,0.201802,1.0,0.213527
2,NM_001003803,ATP5S,81.0,30.214920,83.0,37.193868,71.0,43.258655,46.0,24.625447,26.0,14.727463
3,NM_001003800,BICD2,501.0,47.157866,389.0,43.986848,284.0,43.662942,257.0,34.716778,230.0,32.874747
4,NM_016649,ESF1,69.0,6.525288,52.0,5.907596,41.0,6.333047,26.0,3.528692,26.0,3.733723


In [29]:
X = np.log2(df[['cdRPKM0','cdRPKM1','cdRPKM2','cdRPKM3','cdRPKM4']].values).astype(int)

In [30]:
print(X.shape)

(11745, 5)


In [31]:
LENGTH, DIMENSION = X.shape

#### Digitize the data

In [32]:
bin_size = 20 #state number of bins here, multiple of 5
step = (np.max(X)-np.min(X))/bin_size
bins = np.arange(np.min(X),np.max(X)+0.1,step)
print(bins)
X = np.digitize(X,bins)

[-5.  -4.1 -3.2 -2.3 -1.4 -0.5  0.4  1.3  2.2  3.1  4.   4.9  5.8  6.7
  7.6  8.5  9.4 10.3 11.2 12.1 13. ]


In [33]:
print(X[:5])

[[10  9  9  9  9]
 [ 6  5  5  4  4]
 [10 12 12 10  9]
 [12 12 12 12 12]
 [ 8  8  8  7  7]]


### Sample network

In [34]:
model = BayesianNetwork.from_samples(X,algorithm='chow-liu')

In [35]:
model.structure

((), (0,), (1,), (2,), (3,))

### Perform EM

In [38]:
path = '/Users/akankshitadash/Desktop/Bayesian Networks1/RPF_chrE/'
for k in range(4,7):
    em(k,X,path)

**************** K = 4 ************************
Length of model 1 : 2976
Length of model 2 : 2876
Length of model 3 : 2977
Length of model 4 : 2916
Initial Model Structures [0.2533844189016603, 0.24487015751383567, 0.2534695615155385, 0.2482758620689655]
((), (0,), (1,), (2,), (3,))
((), (0,), (1,), (2,), (3,))
((), (0,), (1,), (2,), (3,))
((), (0,), (1,), (2,), (3,))
Initial Alphas [0.2533844189016603, 0.24487015751383567, 0.2534695615155385, 0.2482758620689655]
Initial CML -82760.44196981448
CML is -68801.07410876348
CML is -67419.76933131146
CML is -67234.04284827254
CML is -67210.79543934936
CML is -67209.88325329247
3601 3601 3601
1800 1800 1800
3891 3891 3891
2453 2453 2453
**************** K = 5 ************************
Length of model 1 : 2303
Length of model 2 : 2385
Length of model 3 : 2337
Length of model 4 : 2312
Length of model 5 : 2408
Initial Model Structures [0.19608343976160067, 0.20306513409961685, 0.19897828863346104, 0.1968497232865049, 0.2050234142188165]
((), (0,)